In [ ]:
!pip install llama_index

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install --upgrade sqlalchemy

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Now you can access the OpenAI API key using the environment variable:
os.environ.get('OPENAI_API_KEY')

In [5]:
from llama_index import SimpleDirectoryReader
from llama_index import download_loader

# Downloads the document to be used in the chatbot
SimpleDirectoryReader = download_loader("SimpleDirectoryReader")
loader = SimpleDirectoryReader('./data', recursive=True, exclude_hidden=True)
documents = loader.load_data()

In [6]:
# Import necessary classes from the LlamaIndex library
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
# Import to OpenAI class from langchain library
from langchain import OpenAI

# Defines the language model for the LLMPredictor
# In this case, it is an OpenAI template with temperature 0 and name "text-davinci-003".
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003"))

In [7]:
# sets PromptHelper parameters
# The maximum input size for the prompt is 4096 characters
# The maximum output number is 256 tokens
# The maximum chunk overlap is 20 characters
max_input_size = 4096
num_output = 256
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [8]:
# Defines the ServiceContext to encapsulate the prediction and prompt logic
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

# Creates a GPTSimpleVectorIndex from the documents to be indexed
# Uses LLMPredictor text prediction to create a simple vector representation of the documents
# Uses the ServiceContext to encapsulate the prediction and prompt logic
index = GPTVectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [ ]:
from langchain.callbacks import get_openai_callback

# Creates the query engine
query_engine = index.as_query_engine()

# Make the query using the example query string
with get_openai_callback() as cb:
    query_str = "How can I open the tailgate?"
    response = query_engine.query(query_str)
    print(cb)


In [ ]:
formatted_response = response.response.replace('\n\n', '\n')
print(formatted_response)